<a href="https://colab.research.google.com/github/lamarock61/hello-world/blob/master/Kraken_bot_ksm_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install SimpleJSON;

Import library

In [ ]:
#import
import pandas as pd
import requests
import json
import time
import os
import urllib.parse
import hashlib
import hmac
import base64
import json
import math



In [ ]:
# initial paremeters
min_buy_price = 8.40;
amount_to_buy = 1;
max_ksm_amount_to_sell = 0.10;
max_ksm_amount_to_buy = 0.05;
min_ksm_amount_in_folio = 0.01;
min_dot_amount_in_folio = 0.01;
dot_amount_in_folio = 0.0000;
ksm_amount_in_folio = 0.0000;
delta = 0.09;
max_sell_price = round(min_buy_price*(1+delta),2);
trade_fees_percentage = 0.001;
print("min buy price : " + str(min_buy_price));
print("min amount to buy : " + str(amount_to_buy));
print("max sell price : " + str(max_sell_price));
print("trading fees : " + str(trade_fees_percentage*100.000) + "%");
API_key = "xxx";
Private_key = "xxx";

min buy price : 8.4
min amount to buy : 1
max sell price : 9.16
trading fees : 0.1%


In [ ]:
# dynamic parameters
current_ksm_dpt_price = 0;
dot_amount_in_folio = 0;
ksm_amount_in_folio = 0.2379257;

In [ ]:
## get ksm/dot value from kraken
# a = ask
# b = bid
# c = last trade closed
# v = volume
# p = volume weighted average price
# t = number of trades
# l = low
# h = high
# o = today's opening price

resp = requests.get('https://api.kraken.com/0/public/Ticker?pair=KSMDOT')

#print(resp.json())


jsonResponse_current_ksm_dot_price = resp.json();

current_ksm_dot_price = jsonResponse_current_ksm_dot_price["result"]["KSMDOT"]["c"][0];

print("current ksm/dot price : " + current_ksm_dot_price);
print(jsonResponse_current_ksm_dot_price);

current ksm/dot price : 8.30000
{'error': [], 'result': {'KSMDOT': {'a': ['8.29000', '1', '1.000'], 'b': ['8.28000', '11', '11.000'], 'c': ['8.30000', '0.02125404'], 'v': ['2.17408403', '1190.73241859'], 'p': ['8.29536', '8.48021'], 't': [4, 566], 'l': ['8.29000', '8.23000'], 'h': ['8.30000', '8.69000'], 'o': '8.30000'}}}


In [ ]:
print(jsonResponse_current_ksm_dot_price["result"]["KSMDOT"]['c'])

['8.30000', '0.02125404']


In [ ]:
# authenticate to kraken

def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [ ]:
# get ksm & dot amount in folio

# Read Kraken API key and secret stored in environment variables
api_url = "https://api.kraken.com"
#api_key = os.environ['API_KEY_KRAKEN']
api_key = API_key;
#api_sec = os.environ['API_SEC_KRAKEN']
api_sec = Private_key

# Attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

# Construct the request and print the result
resp = kraken_request('/0/private/Balance', {
    "nonce": str(int(1000*time.time()))
}, api_key, api_sec)

#print(resp.json())


jsonResponse_folio_data = resp.json();

try :
  dot_amount_in_folio = float(jsonResponse_folio_data["result"]["DOT"])
except :
  dot_amount_in_folio = 0.0000;

try :
  ksm_amount_in_folio = float(jsonResponse_folio_data["result"]["KSM"])
except :
  ksm_amount_in_folio = 0.0000;

print(ksm_amount_in_folio)




0.0275377


In [ ]:
# get list of open orders
resp = kraken_request('/0/private/OpenOrders', {
    "nonce": str(int(1000*time.time())),
    "trades": True
}, api_key, api_sec)

jsonResponse_open_orders = resp.json();

# get list of open order keys and count keys
list_keys = [];
list_keys = list(jsonResponse_open_orders["result"]["open"].keys())
keys_qty = len(list_keys)
print(keys_qty)


print(jsonResponse_open_orders["result"]["open"]);
# print(list(jsonResponse_open_orders["result"]["open"].keys()));
# print(jsonResponse_open_orders["result"]["open"]["ON66J4-FNUI2-4BXA3R"]["descr"]["pair"]);

if keys_qty == 0:
  sell_orders_qty = 0
  buy_orders_qty = 0

open_order = jsonResponse_open_orders["result"]["open"]
for _key in list_keys:
  if open_order[_key]["descr"]["pair"] == "KSMDOT" and open_order[_key]["descr"]["type"] == "sell":
    sell_orders_qty = 1
    # sell order allready exists
  else:
    sell_orders_qty = 0

for _key in list_keys:
  if open_order[_key]["descr"]["pair"] == "KSMDOT" and open_order[_key]["descr"]["type"] == "buy":
    buy_orders_qty = 1
    # buy order already exists
  else:
    buy_orders_qty = 0

# create sell order if needed
if sell_orders_qty == 0 and dot_amount_in_folio < min_dot_amount_in_folio:
        # place new sell order
  print("place new sell order")
  resp = kraken_request('/0/private/AddOrder', {
      "nonce": str(int(1000*time.time())),
      "ordertype": "limit",
      "type": "sell",
      "volume": min(ksm_amount_in_folio, max_ksm_amount_to_sell),
      "pair": "KSMDOT",
      "price": max_sell_price
  }, api_key, api_sec)

  print(resp.json())

# create buy order if needed
if buy_orders_qty == 0 and ksm_amount_in_folio < min_ksm_amount_in_folio:
  # place new buy order
  print("place new buy order")
  resp = kraken_request('/0/private/AddOrder', {
      "nonce": str(int(1000*time.time())),
      "ordertype": "limit",
      "type": "buy",
      "volume": min(dot_amount_in_folio, max_ksm_amount_to_buy),
      "pair": "KSMDOT",
      "price": min_buy_price
  }, api_key, api_sec)

  print(resp.json())



'''
    if dot_amount_in_folio < min_dot_amount_in_folio:
      print("place new buy order")
      # place new buy order
'''
#print(jsonResponse_open_orders["result"]["open"][_key]["descr"])
print("sell order qty : " + str(sell_orders_qty))
print("buy order qty : " + str(buy_orders_qty))

1
{'O65QGD-SOSIU-LB3IYM': {'refid': None, 'userref': 0, 'status': 'open', 'opentm': 1635550728.6913, 'starttm': 0, 'expiretm': 0, 'descr': {'pair': 'KSMDOT', 'type': 'sell', 'ordertype': 'limit', 'price': '9.16', 'price2': '0', 'leverage': 'none', 'order': 'sell 0.02753770 KSMDOT @ limit 9.16', 'close': ''}, 'vol': '0.02753770', 'vol_exec': '0.00000000', 'cost': '0.00000', 'fee': '0.00000', 'price': '0.00000', 'stopprice': '0.00000', 'limitprice': '0.00000', 'misc': '', 'oflags': 'fciq'}}
sell order qty : 1
buy order qty : 0
